# Project 3 - Wrangle OpenStreetMap New York Metropolitan Area

Christian Guzman

June 2017

In [1]:
import csv
import codecs
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict
import os

import cerberus

import schema

In [2]:
#Main OSM
OSM_FILE = "C:/Users/cguzm_000/Desktop/Desk Docs/Data/newyork.osm"
#k = 500

In [3]:
osm_file = "C:/Users/cguzm_000/Documents/udacity-projects/data/sample.osm"

SCHEMA = schema.schema
NODES_PATH = "data/nodes.csv"
NODE_TAGS_PATH = "data/nodes_tags.csv"
WAYS_PATH = "data/ways.csv"
WAY_NODES_PATH = "data/ways_nodes.csv"
WAY_TAGS_PATH = "data/ways_tags.csv"

In [4]:
def count_tags(file):
    unique_tags = defaultdict(int)
    for event, elem in ET.iterparse(file, events=("start",)):
        unique_tags[elem.tag] += 1
    return unique_tags

In [5]:
count_tags(osm_file)

defaultdict(int,
            {'member': 174,
             'nd': 29795,
             'node': 22975,
             'osm': 1,
             'relation': 19,
             'tag': 19426,
             'way': 3599})

In [6]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        ######
        if lower.search(element.attrib['k']):
            #print(element.attrib['k'])
            keys["lower"] += 1
        elif lower_colon.search(element.attrib['k']):
            #print(element.attrib['k'])
            keys["lower_colon"] += 1
        elif problemchars.search(element.attrib['k']):
            #print(element.attrib['k'])
            keys["problemchars"] += 1
        else:
            #print(element.attrib['k'])
            keys["other"] += 1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [7]:
process_map(osm_file)

{'lower': 7525, 'lower_colon': 11664, 'other': 189, 'problemchars': 48}

In [8]:
#EXPLORING USERS
def get_user(element):
    return element.attrib.get('uid')


def process_users(filename):
    users = [] #set()
    for _, elem in ET.iterparse(filename, events=("start",)):
        if get_user(elem):
            users.append(get_user(elem))
    users = set(users)
        
    return users

In [9]:
process_users(osm_file)

{'100643',
 '1012362',
 '102691',
 '103253',
 '104519',
 '104962',
 '1051550',
 '105609',
 '105839',
 '105946',
 '1072240',
 '1087647',
 '109276',
 '1093326',
 '109362',
 '1110270',
 '111159',
 '113450',
 '115611',
 '11617',
 '1168086',
 '118134',
 '1185712',
 '118871',
 '119748',
 '119881',
 '120146',
 '1202134',
 '121241',
 '1219875',
 '1221143',
 '1221149',
 '1221152',
 '12290',
 '1231593',
 '1232894',
 '123364',
 '123633',
 '1240849',
 '1240864',
 '12448',
 '125718',
 '12581',
 '126442',
 '1289164',
 '129614',
 '130472',
 '1306',
 '1311287',
 '13203',
 '1323363',
 '1323730',
 '132803',
 '1337142',
 '134892',
 '135329',
 '1357992',
 '1376118',
 '1380816',
 '1384038',
 '13944',
 '139856',
 '1408522',
 '141082',
 '1424765',
 '1426068',
 '14293',
 '143002',
 '146680',
 '147510',
 '147851',
 '148169',
 '1484939',
 '14850',
 '148800',
 '1494110',
 '150272',
 '150368',
 '150692',
 '1520225',
 '152074',
 '153669',
 '154730',
 '155262',
 '158826',
 '1597155',
 '160042',
 '160949',
 '161619'

In [10]:
#IMPROVING STREET NAMES
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place",
            "Square", "Lane", "Road", "Trail", "Parkway", "Commons"]
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In [11]:
pprint.pprint(dict(audit(osm_file)))

{'10': set(['Route 10']),
 '27': set(['NJ 27']),
 'A': set(['Avenue A']),
 'Ave': set(['Park Ave']),
 'B': set(['Avenue B']),
 'Broadway': set(['Broadway', 'East Broadway', 'West Broadway']),
 'Circle': set(['64th Circle', 'Alex Circle', 'Covington Circle']),
 'Concourse': set(['Grand Concourse']),
 'Crescent': set(['246th Crescent',
                  'Asquith Crescent',
                  'Burden Crescent',
                  'Cromwell Crescent']),
 'E': set(['Bay Drive E', 'Lakeview Avenue E', 'Locust Avenue E']),
 'East': set(['Belmar Drive East',
              'Clearview Expressway Service Road East',
              'Drumgoole Road East',
              'Park Drive East']),
 'Extension': set(['Eastern Parkway Extension']),
 'Green': set(['Carlyle Green', 'Forest Green']),
 'Heights': set(['Columbia Heights']),
 'Highway': set(['Kings Highway', 'Montauk Highway']),
 'Hl': set(['Diana Hl']),
 'K': set(['Avenue K']),
 'L': set(['Avenue L']),
 'Loop': set(['Bethel Loop',
              'Mar

In [12]:
mapping = { "St": "Street",
            "St.": "Street",
            "Rd": "Road",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "N": "North",
            "S": "South",
            "E": "East",
            "W": "West",
            "Hl": "Hill",
            "Ter": "Terrace",
            "Pkwy": "Parkway",
            "Sr": "State Road"
           }

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m and m.group() in mapping.keys():
        name = street_type_re.sub(mapping[m.group()], name)
    return name

In [13]:
def test():
    for st_type, ways in audit(osm_file).iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            if name is not better_name:
                print name, "=>", better_name
test()

Diana Hl => Diana Hill
Lido Pkwy => Lido Parkway
Granada Pkwy => Granada Parkway
Locust Avenue E => Locust Avenue East
Lakeview Avenue E => Lakeview Avenue East
Bay Drive E => Bay Drive East
Spur Drive N => Spur Drive North
Salisbury Drive N => Salisbury Drive North
Avenue N => Avenue North
Horace Harding Expressway Sr S => Horace Harding Expressway Sr South
Avenue S => Avenue South
Parkway Drive S => Parkway Drive South
Somerset Road W => Somerset Road West
Bayview Avenue W => Bayview Avenue West
Avenue W => Avenue West
Park Ave => Park Avenue
Grand St => Grand Street
McKinley Ter => McKinley Terrace


## Updating "Sr" to "State Road"

Some of the adresses contained the abbreviation **Sr** which stands for **State Road**.

In [14]:
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

sr_re = re.compile(r'\bSr\b')
def update_name(name, mapping):
    m = street_type_re.search(name)
    state_road = sr_re.search(name)
    if m and m.group() in mapping.keys():
        name = street_type_re.sub(mapping[m.group()], name)
    if state_road:
        name = sr_re.sub(mapping[state_road.group()], name)
    return name


for element in get_element(osm_file, tags=('node','way')):
    for tag in element.iter("tag"):
        if tag.attrib['k'] == "addr:street":
            print update_name(tag.attrib['v'], mapping)
            

Grand Street
Amsterdam Avenue
West 13th Street
Fulton Street
West 23rd Street
Gordon Street
Grandview Avenue
Stratford Road
Grand Street
De Graw Street
Henry Street
Clinton Street
3rd Place
Montgomery Avenue
Dahill Road
Cortelyou Road
26th Avenue
Maryland Avenue
Hastings Street
Castleton Avenue
Meagan Loop
Richmond Terrace
East 53rd Street
Avenue K
Greenleaf Avenue
East 38th Street
Caswell Avenue
Marc Street
Maple Parkway
East 53rd Street
Foster Avenue
Debbie Street
Elmwood Park Drive
Drew Street
Latourette Lane
Etna Street
Logan Street
233rd Street
264th Street
251st Street
Union Turnpike
Douglaston Parkway
Rose Avenue
Broadway
184th Street
Spring Street
Marcus Garvey Boulevard
Congress Street
De Kalb Avenue
President Street
Greenwich Avenue
Hillside Avenue
Nostrand Avenue
Pacific Street
Hillside Avenue
Browvale Lane
East 3rd Street
Timber Ridge Drive
147th Street
Drumgoole Road East
Neptune Avenue
Union Turnpike
Brighton 4th Street
83rd Street
64th Circle
Horace Harding Expressway St

In [15]:
a = 2
b = 0
if a == 2:
    b += 1
    print 'a is 2'
if a % 2 == 0:
    b = 100
    print 'a is divisible by 2'
print b

a is 2
a is divisible by 2
100


In [16]:
def test():
    for st_type, ways in audit(osm_file).iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            if name is not better_name:
                print name, "=>", better_name

test()

Diana Hl => Diana Hill
Lido Pkwy => Lido Parkway
Granada Pkwy => Granada Parkway
Locust Avenue E => Locust Avenue East
Lakeview Avenue E => Lakeview Avenue East
Bay Drive E => Bay Drive East
Spur Drive N => Spur Drive North
Salisbury Drive N => Salisbury Drive North
Avenue N => Avenue North
Horace Harding Expressway Sr S => Horace Harding Expressway State Road South
Avenue S => Avenue South
Parkway Drive S => Parkway Drive South
Somerset Road W => Somerset Road West
Bayview Avenue W => Bayview Avenue West
Avenue W => Avenue West
Park Ave => Park Avenue
Grand St => Grand Street
Grand Central Parkway Sr South => Grand Central Parkway State Road South
McKinley Ter => McKinley Terrace


## Auditing Postal Codes

In [17]:
postal_re = re.compile(r'\d{5}', re.IGNORECASE)

def key_type(element, keys):
    if element.attrib['k'] == "addr:postcode":
        if postal_re.search(element.attrib['v']):
            #print(element.attrib['v'])
            keys["5-digit postal"] += 1
        else:
            #print(element.attrib['v'])
            keys["other"] += 1
    return keys

def process_map(filename):
    keys = {"5-digit postal": 0, "other": 0}
    for element in get_element(filename):
        for tag in element.iter("tag"):
            keys = key_type(tag, keys)

    return keys
process_map(osm_file)

def update_postal(code):
    code.split('')

In [18]:
s = '123RANDOM45WALK456'
def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

def update_post(code):
    better_code = ''
    for e in list(code):
        if is_number(e):
            better_code += e
            if len(better_code) == 5:
                break
    return better_code

In [19]:
update_post(s)

'12345'

## Auditing City Name

In [20]:
cities = defaultdict(int)
for element in get_element(osm_file):
    if element.tag == "node" or "way":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "addr:city":
                cities[tag.attrib['v']] += 1
pprint.pprint(dict(cities))
                

{'Amityville': 3,
 'Asharoken': 1,
 'Babylon': 6,
 'Bay Shore': 12,
 'Baywood': 3,
 'Brentwood': 15,
 'Brightwaters': 4,
 'Brooklyn': 5,
 'Centerport': 4,
 'Central Islip': 1,
 'Chatham': 1,
 'Clifton': 1,
 'Cold Spring Harbor': 5,
 'Commack': 24,
 'Copiague': 16,
 'Deer Park': 18,
 'Dix Hills': 11,
 'East Brunswick': 2,
 'East Farmingdale': 4,
 'East Northport': 14,
 'Eatons Neck': 1,
 'Edison': 1,
 'Elmsford': 1,
 'Elwood': 5,
 'Fort Salonga': 8,
 'Greenlawn': 7,
 'Halesite': 4,
 'Hauppauge': 3,
 'Hoboken': 2,
 'Huntington': 14,
 'Huntington Station': 16,
 'Irvington': 1,
 'Islip': 1,
 'Kenilworth': 1,
 'Kings Park': 7,
 'Levittown': 2,
 'Lindenhurst': 15,
 'Lloyd Harbor': 3,
 'Locust Valley': 2,
 'Melville': 6,
 'Montclair': 1,
 'New York': 14,
 'New York City': 1,
 'North Amityville': 6,
 'North Babylon': 9,
 'North Bay Shore': 9,
 'North Lindenhurst': 7,
 'Northport': 3,
 'Parsippany': 1,
 'Piscataway': 1,
 'Riverdale': 1,
 'Rye': 1,
 'Smithtown': 7,
 'South Huntington': 3,
 'West

In [21]:
city_mapping = ['New York City', 'Brooklyn', 'Bronx', 'Queens', 'Staten Island']
def update_city_name(name, mapping):
    if name in mapping:
        name = 'New York'
    return name


for element in get_element(osm_file, tags=('node','way')):
    for tag in element.iter("tag"):
        if tag.attrib['k'] == "addr:city" and tag.attrib['v'] in city_mapping:
            better_name = update_city_name(tag.attrib['v'], mapping=city_mapping)
            print tag.attrib['v'],"=>", better_name

Brooklyn => New York
Brooklyn => New York
New York City => New York
Brooklyn => New York
Brooklyn => New York
Brooklyn => New York


In [22]:
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version',
               'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset',
              'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS,
                  way_attr_fields=WAY_FIELDS, problem_chars=PROBLEMCHARS,
                  default_tag_type='regular'):
    '''Clean and shape node or way XML element to python dict'''

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = [] # Handle secondary tags the same way for both node and way elements
    # MY CODE HERE
    if element.tag == 'node':
        for k in element.attrib:
            if k in node_attr_fields:
                node_attribs[k] = element.attrib[k]
        if len(element.findall('tag')) > 0:
            for tag in element.iter('tag'):
                problem = PROBLEMCHARS.search(tag.attrib['k'])
                if problem:
                    print(problem.group())
                    continue
                tag_dict = {}
                tag_dict['id'] = element.attrib['id']
                
                # Dealing with colons
                m = LOWER_COLON.search(tag.attrib['k'])
                if m:
                    key_split = tag.attrib['k'].split(":")
                    key = ':'.join(key_split[1:])
                    tag_dict['key'] = key
                    tag_dict['type'] = key_split[0]
                else:
                    tag_dict['key'] = tag.attrib['k']
                    tag_dict['type'] = default_tag_type
                #UPDATING STREETNAMES
                if tag.attrib['k'] == "addr:street":
                    tag_dict['value'] = update_name(tag.attrib['v'], mapping)
                #UPDATING CITY NAMES
                elif tag.attrib['k'] == "addr:city":
                    tag_dict['value'] = update_city_name(tag.attrib['v'], city_mapping)
                #UPDATING POSTCODES
                elif tag.attrib['k'] == "addr:postcode":
                    tag_dict['value'] = update_post(tag.attrib['v'])
                else:
                    tag_dict['value'] = tag.attrib['v']
                    
                tags.append(tag_dict)
        
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for k in element.attrib:
            if k in way_attr_fields:
                way_attribs[k] = element.attrib[k]
        if len(element.findall('tag')) > 0:
            for tag in element.iter('tag'):
                problem = PROBLEMCHARS.search(tag.attrib['k'])
                if problem:
                    print(problem.group())
                    continue
                tag_dict = {}
                tag_dict['id'] = element.attrib['id']
                # Dealing with colons
                m = LOWER_COLON.search(tag.attrib['k'])
                if m:
                    key_split = tag.attrib['k'].split(":")
                    key = ':'.join(key_split[1:])
                    tag_dict['key'] = key
                    tag_dict['type'] = key_split[0]
                else:
                    tag_dict['key'] = tag.attrib['k']
                    tag_dict['type'] = default_tag_type
                #UPDATING STREETNAMES
                if tag.attrib['k'] == "addr:street":
                    tag_dict['value'] = update_name(tag.attrib['v'], mapping)
                #UPDATING CITY NAMES
                elif tag.attrib['k'] == "addr:city":
                    tag_dict['value'] = update_city_name(tag.attrib['v'], city_mapping)
                #UPDATING POSTCODES
                elif tag.attrib['k'] == "addr:postcode":
                    tag_dict['value'] = update_post(tag.attrib['v'])
                else:
                    tag_dict['value'] = tag.attrib['v']
                    
                tags.append(tag_dict)
        if len(element.findall('nd')) > 0:
            position = 0
            for nd in element.iter('nd'):
                node_dict = {}
                node_dict['id'] = element.attrib['id']
                node_dict['node_id'] = nd.attrib['ref']
                node_dict['position'] = position
                way_nodes.append(node_dict)
                position += 1
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

def test():
    i = 0
    for element in get_element(osm_file, tags=('node', 'way')):
        el = shape_element(element)
        pprint.pprint(el)
        i += 1
        if i == 2000:
            break

In [23]:
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [24]:
def main_function(file_in, validate):
    '''Iteratively process each XML element'''
    
    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH,'w') as way_tags_file:
             
         nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
         node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
         ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
         way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
         way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

         nodes_writer.writeheader()
         node_tags_writer.writeheader()
         ways_writer.writeheader()
         way_nodes_writer.writeheader()
         way_tags_writer.writeheader()
         
         validator = cerberus.Validator()
         
         for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                    
if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    main_function(osm_file, validate=True)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


## SQL PART

In [24]:
import sqlite3

In [125]:
conn = sqlite3.connect("nymetro.db")
cursor = conn.cursor()
#query1 = cursor.execute('''USE nymetro''')
tables = ['nodes', 'nodes_tags', 'ways', 'ways_nodes', 'ways_tags']
for t in tables:
    cursor.execute('DROP TABLE IF EXISTS {}'.format(t))
#   cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
#   tables=(cursor.fetchall())
#   print tables
cursor.execute('''CREATE TABLE IF NOT EXISTS 
                    nodes(
                    id INT PRIMARY KEY NOT NULL,
                    lat FLOAT, lon FLOAT, 
                    user VARCHAR, uid INT, 
                    version INT, changeset INT, 
                    timestamp DATETIME);''')
cursor.execute('''CREATE TABLE IF NOT EXISTS
                    nodes_tags(
                    id INT NOT NULL,
                    key VARCHAR, value VARCHAR, 
                    type VARCHAR);''')
cursor.execute('''CREATE TABLE IF NOT EXISTS
                    ways(
                    id INT PRIMARY KEY NOT NULL, user VARCHAR, 
                    uid INT, version INT, changeset INT, 
                    timestamp DATETIME);''')
cursor.execute('''CREATE TABLE IF NOT EXISTS
                    ways_nodes(
                    id INT NOT NULL, 
                    node_id INT, position INT);''')
cursor.execute('''CREATE TABLE IF NOT EXISTS
                    ways_tags(
                    id INT NOT NULL, 
                    key VARCHAR, value VARCHAR, 
                    type VARCHAR);''')
conn.commit()

In [128]:
cursor.execute('SELECT COUNT(*) FROM ways;')
results = cursor.fetchall()
print results

[(3599,)]


In [127]:
# Read in the csv file as a dictionary, format the
# data as a list of tuples:
for t in tables:
    with open('./data/{}.csv'.format(t), 'rb') as f:
        dr = csv.DictReader(f) # comma is default delimiter
        if t == 'nodes':
            to_db = [(i['id'].decode("utf-8"), i['lat'].decode("utf-8"), i['lon'].decode("utf-8"), i['user'].decode("utf-8"), 
                      i['uid'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"),
                     i['timestamp'].decode("utf-8")) for i in dr]
        # insert the formatted data
            cursor.executemany('''INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp)
                            VALUES(?, ?, ?, ?, ?, ?, ?, ?);''', to_db)
            conn.commit()
        elif t == 'nodes_tags':
            to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), 
                      i['type'].decode("utf-8")) for i in dr]
            cursor.executemany('''INSERT INTO nodes_tags(id, key, value, type)
                            VALUES(?, ?, ?, ?);''', to_db)
            conn.commit()
        elif t == 'ways':
            to_db = [(i['id'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['version'].decode("utf-8"), 
                     i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8")) for i in dr]
            cursor.executemany('''INSERT INTO ways(id, user, uid, version, changeset, timestamp)
                            VALUES(?, ?, ?, ?, ?, ?);''', to_db)
            conn.commit()
        elif t == 'ways_nodes':
            to_db = [(i['id'].decode("utf-8"), i['node_id'].decode("utf-8"), 
                      i['position'].decode("utf-8")) for i in dr]
            cursor.executemany('''INSERT INTO ways_nodes(id, node_id, position)
                            VALUES(?, ?, ?);''', to_db)
            conn.commit()
        elif t == 'ways_tags':
            to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), 
                      i['type'].decode("utf-8")) for i in dr]
            cursor.executemany('''INSERT INTO ways_tags(id, key, value, type)
                            VALUES(?, ?, ?, ?);''', to_db)
            conn.commit()
            
    

In [39]:
cursor.execute('SELECT * FROM nodes LIMIT 10;')
all_rows = cursor.fetchall()
print('1):')
pprint.pprint(all_rows, width=150, indent=2)

1):
[ (26769789, 40.6995927, -74.1868914, u'wambag', 326503, 4, 41015803, u'2016-07-25T17:17:46Z'),
  (42423468, 40.723915, -74.0064737, u'SH17', 2591427, 4, 34666473, u'2015-10-16T00:30:14Z'),
  (42427925, 40.8738008, -73.9086174, u'WestsideGuy', 716239, 3, 16104998, u'2013-05-13T02:17:25Z'),
  (42430320, 40.752935, -73.985488, u'NJ Engineer', 703728, 3, 12104263, u'2012-07-03T20:26:08Z'),
  (42432970, 40.825956, -73.943357, u'Eliyak', 439046, 3, 18485081, u'2013-10-22T12:01:57Z'),
  (42435470, 40.719324, -73.997303, u'NE2', 207745, 4, 3381786, u'2009-12-15T18:21:16Z'),
  (42437992, 40.716235, -73.992014, u'NE2', 207745, 3, 3381786, u'2009-12-15T18:21:24Z'),
  (42440785, 40.7144101, -73.9902458, u'dc157', 1714546, 6, 17792753, u'2013-09-12T02:08:29Z'),
  (42443712, 40.7833517, -73.9231593, u'woodpeck_fixbot', 147510, 8, 2221591, u'2009-08-21T10:59:56Z'),
  (42447275, 40.7954638, -73.9638068, u'NE2', 207745, 3, 3396277, u'2009-12-17T21:44:02Z')]


## CHECK IF AUDITS WERE GOOD
1. postcodes
2. city
3. state
4. county_name

In [75]:
def countQuery(key, limit=10):
    cursor.execute('''SELECT tags.value, COUNT(*) AS total FROM
                        (SELECT * FROM nodes_tags 
                        UNION ALL
                        SELECT * FROM ways_tags) AS tags
                        WHERE tags.key = '{}'
                        GROUP BY tags.value 
                        ORDER BY total DESC
                        LIMIT {};'''.format(key, limit))
    results = cursor.fetchall()
    return results

In [76]:
countQuery(key='postcode')

[(u'10314', 49),
 (u'11234', 38),
 (u'10312', 35),
 (u'11203', 31),
 (u'11236', 31),
 (u'11746', 31),
 (u'11385', 29),
 (u'11706', 28),
 (u'10469', 27),
 (u'11743', 27)]

In [77]:
countQuery('city')

[(u'West Babylon', 26),
 (u'Commack', 24),
 (u'New York', 20),
 (u'West Islip', 19),
 (u'Deer Park', 18),
 (u'Copiague', 16),
 (u'Huntington Station', 16),
 (u'Brentwood', 15),
 (u'Lindenhurst', 15),
 (u'East Northport', 14)]

In [80]:
countQuery('state')

[(u'NY', 22), (u'NJ', 7), (u'Nj', 1)]

In [81]:
countQuery('county')

[(u'Nassau, NY', 48),
 (u'Bergen, NJ', 29),
 (u'Middlesex, NJ', 27),
 (u'Queens, NY', 26),
 (u'Westchester, NY', 24),
 (u'Essex, NJ', 21),
 (u'Suffolk, NY', 20),
 (u'Kings, NY', 16),
 (u'Passaic, NJ', 15),
 (u'Bronx, NY', 13)]

## Overview Statistics of Data

## File sizes

In [89]:
#Show files and their sizes:

dirpath = "C:/Users/cguzm_000/Documents/udacity-projects/OpenStreetMapProject_Files"

files_list = []
for path, dirs, files in os.walk(dirpath):
    files_list.extend([(filename, os.path.getsize(os.path.join(path, filename))) for filename in files])
for filename, size in files_list:
    if size < 1*10**6:
        print '{:.<40s}: {:0d}KB'.format(filename,size/1000)
    else:
        print '{:.<40s}: {:0d}MB'.format(filename,size/1000000)

nodes.csv...............................: 2MB
nodesTest.csv...........................: 0KB
nodes_tags.csv..........................: 58KB
nymetro.db..............................: 3MB
sample.osm..............................: 5MB
ways.csv................................: 237KB
ways_nodes.csv..........................: 744KB
ways_tags.csv...........................: 603KB


## Number of nodes

In [99]:
def printQuery(query, all=False):
    query
    if all:
        pprint.pprint(cursor.fetchall())
    else:
        print cursor.fetchall()[0][0]

query = cursor.execute('''SELECT COUNT(*) FROM nodes;''')
printQuery(query)

22975


## Number of ways

In [66]:
query = cursor.execute('''SELECT COUNT(*) FROM ways;''')
printQuery(query)

3599


## Number of unique users

In [70]:
query = cursor.execute('''SELECT COUNT(DISTINCT(uniques.uid)) FROM (SELECT uid FROM nodes
                  UNION
                  SELECT uid FROM ways) AS uniques;
                    ''')
printQuery(query)

645


## Top 10 contributing users

In [131]:
query = cursor.execute('''SELECT elems.user, COUNT(*) as total
                        FROM (SELECT user FROM nodes 
                        UNION ALL 
                        SELECT user FROM ways) elems
                        GROUP BY elems.user
                        ORDER BY total DESC
                        LIMIT 10;''')

printQuery(query, all=True)

[(u'Rub21_nycbuildings', 9753),
 (u'ingalls_nycbuildings', 1885),
 (u'MySuffolkNY', 1254),
 (u'woodpeck_fixbot', 1242),
 (u'SuffolkNY', 1163),
 (u'minewman', 1004),
 (u'Northfork', 827),
 (u'ediyes_nycbuildings', 543),
 (u'lxbarth_nycbuildings', 471),
 (u'smlevine', 439)]


## Top ten amenities

In [109]:
countQuery(key='amenity', limit=10)

[(u'parking', 16),
 (u'bicycle_parking', 10),
 (u'restaurant', 9),
 (u'place_of_worship', 7),
 (u'school', 5),
 (u'grave_yard', 3),
 (u'bank', 2),
 (u'cafe', 2),
 (u'fast_food', 2),
 (u'fire_station', 2)]

## Places of leisure

In [107]:
countQuery(key='leisure', limit=100)

[(u'park', 13),
 (u'pitch', 13),
 (u'garden', 6),
 (u'playground', 3),
 (u'swimming_pool', 2),
 (u'sports_centre', 1)]

## Types of shops

In [111]:
countQuery(key='shop', limit=10)

[(u'clothes', 3),
 (u'gift', 2),
 (u'bakery', 1),
 (u'car_repair', 1),
 (u'confectionery', 1),
 (u'convenience', 1),
 (u'funeral_directors', 1),
 (u'mall', 1),
 (u'mobile_phone', 1),
 (u'newsagent', 1)]

## Sports

In [113]:
countQuery(key='sport', limit=100)

[(u'baseball', 3),
 (u'soccer', 3),
 (u'tennis', 3),
 (u'basketball', 2),
 (u'athletics', 1)]